# Sequence To SQL

Welcome to our project in the Advanced Natural Language Processing course

We try to build it with the data provided in https://github.com/salesforce/WikiSQL

Remove this: https://towardsdatascience.com/text-to-sql-learning-to-query-tables-with-natural-language-7d714e60a70d?gi=6b6c7e91e298

In [ ]:
import torch

if torch.cuda.is_available():
    device = torch.cuda.current_device()
    print('Current device:', torch.cuda.get_device_name(device))
else:
    print('Failed to find GPU. Will use CPU.')
    device = 'cpu'

# Data collection and Review
Clone the data from the WikiSQL git repository and install them.

In [ ]:
# Make sure you are in the /src folder for the next cell to work properly
!git clone https://github.com/salesforce/WikiSQL

In [ ]:
import os

data_folder = '../data'
data_sql_dir = os.path.isdir(data_folder)

if not data_sql_dir:
  %cd ..
  !git clone https://github.com/salesforce/WikiSQL
  %cd WikiSQL
  !tar xvjf data.tar.bz2
  !move data ../
  %cd ..

print("Data available")
print(os.listdir(data_folder))
  

Take a look inside the data


In [ ]:
!head -5 data/dev.jsonl
!head -5 data/dev.tables.jsonl

In [ ]:
import json

def read_json_data_from_file(file: str):
  ret_data = []
  with open(file) as json_file:
    while True:
      # Get next line from file
      line = json_file.readline()
      if not line:
        break

      data = json.loads(line)
      ret_data.append(data)
  return ret_data

def convert_to_id_dict(data, id_key: str):
  ret_dict = {}
  for element in data:
    if id_key in element:
      ret_dict[element[id_key]] = element
    else:
      print(f'Element {element} doenst contain key {id_key}')
  return ret_dict

Lets see if we succesfully serialized the data into objects.

In [ ]:
dev_req_data = read_json_data_from_file('data/dev.jsonl')
dev_table_data = read_json_data_from_file('data/dev.tables.jsonl')
 
print(f'We have {len(dev_req_data)} dev data with {len(dev_table_data)} tables.')
print(f'An example Request: ')
print(json.dumps(dev_req_data[0], indent=2))

### The fields represent the following:

* `phase`: the phase in which the dataset was collected. We collected WikiSQL in two phases.
* `question`: the natural language question written by the worker.
* `table_id`: the ID of the table to which this question is addressed.
sql: the SQL query corresponding to the question. This has the following *subfields:
  * `sel`: the numerical index of the column that is being selected. You can find the actual column from the table.
  * `agg`: the numerical index of the aggregation operator that is being used. You can find the actual operator from Query.agg_ops in lib/query.py.
  * `conds`: a list of triplets (column_index, operator_index, condition) where:
    * `column_index`: the numerical index of the condition column that is being used. You can find the actual column from the table.
    * `operator_index`: the numerical index of the condition operator that is being used. You can find the actual operator from Query.cond_ops in lib/query.py.
    * `condition`: the comparison value for the condition, in either string or float type.



In [ ]:
print(f'An example Table: ')
print(json.dumps(dev_table_data[0], indent=2))

## Preprocess

The data is stored with indices but we need the actual column names so saturate the requests with the data

In [ ]:
# Transform the data into a dictonary index by the id
dev_table_data_dict = convert_to_id_dict(dev_table_data, 'id')

In [ ]:
# Get the preliminary data
# Maybe we want the other idexes also

In [ ]:
def get_table_column(data_list, tables_dict):
  ret_list = []
  for element in data_list:
    current_table = tables_dict[element['table_id']]
    columns = current_table['header']
    # Replace the index
    element['columns'] = columns
    element['sql']['sel'] = columns[element['sql']['sel']]

    # For the where conditions
    for cond in element['sql']['conds']:
      cond[0] = columns[cond[0]]
    ret_list.append(element)
  return ret_list


In [ ]:
dev_req_data = read_json_data_from_file('data/dev.jsonl')
dev_table_data = read_json_data_from_file('data/dev.tables.jsonl')
dev_prep_req_data = get_table_column(dev_req_data, dev_table_data_dict)
print(f'Filed in with the Columns: ')
print(json.dumps(dev_prep_req_data[-2], indent=2))

# Build the model

In [ ]:
!pip install transformers

In this section, we will look into **contextual embeddings**. 

For this we use [**pretrained BERT**](https://www.aclweb.org/anthology/N19-1423.pdf) provided via [HuggingFace](https://huggingface.co/).

Let's first install the HuggingFace python package:

In [ ]:
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = BertModel.from_pretrained("bert-base-cased")

sent = 'I left my phone in my left pocket'

encoded_input = tokenizer(sent, add_special_tokens=True, return_tensors='pt')

with torch.no_grad():
    output = model(**encoded_input)

print(output.last_hidden_state.squeeze(0).shape)

In [ ]:
dev_table_data_dict['2-1123478-2']

In [ ]:
# reference is hydranet https://arxiv.org/pdf/2008.04759.pdf

def get_input_vetor(request):

  input_list = []
  
  c_table = dev_table_data_dict[request['table_id']]

  table_name = c_table['caption'] #should be name not id
  sep_token = '[SEP]'
  space_token = ' '
  cls_token = '[CLS]'
  columns = request['columns']
  question = request['question'] # might need to be tokenized
  for i, col in enumerate(columns):
    col_type = c_table['types'][i] # infere type somehow
    token_seq = cls_token + col_type + space_token + table_name + space_token + col + sep_token + question + sep_token
    embedded_token_seq = token_seq # embedding here ?! 
    input_list.append(embedded_token_seq)

  return input_list

In [ ]:
get_input_vetor(dev_prep_req_data[-2])

In [ ]:
from transformers import BertTokenizer, BertForMultipleChoice

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
[tokenizer.tokenize(input_text) for input_text in get_input_vetor(dev_prep_req_data[-2])]

model = BertForMultipleChoice.from_pretrained("bert-base-cased")



In [ ]:
import torch
import torch.nn as nn
from transformers import MobileBertTokenizer, MobileBertForMultipleChoice

tokenizer = MobileBertTokenizer.from_pretrained('google/mobilebert-uncased')
model = MobileBertForMultipleChoice.from_pretrained('google/mobilebert-uncased')

fact_sent = "Modern pizza was invented in Naples, and the dish and its variants have since become popular in many countries."
answer_1 = "It has become one of the most popular foods in the world"
answer_2 = "It has gone nowhere from there"

labels = torch.tensor(0).unsqueeze(0)
encoding = tokenizer([[fact_sent, fact_sent], [answer_1, answer_2]], return_tensors='pt', padding=True)

outputs = model(**{k: v.unsqueeze(0) for k,v in encoding.items()}, labels=labels)

loss, logits = outputs[:2]

softmax = nn.Softmax(dim=1)
probabilities = softmax(logits)

In [ ]:
probabilities

In [ ]:
from transformers import BertTokenizer
from transformers import BertForTokenClassification
from transformers import BertConfig
import torch.optim as optim

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
[tokenizer.tokenize(input_text) for input_text in get_input_vetor(dev_prep_req_data[-2])]

model = BertForMultipleChoice.from_pretrained("bert-base-cased")

# Show the model configuration
print(model.config)

# Optimer has relatively low learning rate as we are finetuning a trained model.
optimizer = optim.Adam(model.parameters(), lr=2e-5)

training_data = dataset_subword.data['train_subword']

batch_size = 32
validation_batch_size = 16
epochs=1    # Total number of epochs
report_every=5000   # Report validation performance after every X sentences

for epoch in range(1, epochs+1):
    print('Epoch', epoch)
    
    sent_cnt = 0
    text_batch = []
    label_batch = []
    max_len = 0

    for _, targets, full_sent in training_data:
        sent_cnt += 1
        # Accumulate sentences and label sequences for the current batch
        text_batch.append(full_sent)
        label_batch.append(targets)
        max_len = max(max_len, len(targets))

        if len(text_batch) == batch_size or sent_cnt == len(training_data)-1:
            # Tokenize the batch of input sentences
            encoding = tokenizer(text_batch, return_tensors='pt', add_special_tokens=False, padding=True, truncation=True).to(device)
            # Pad the the labels to the maximum length in the current batch.
            # (No need to pad input sentences as that's handled in the tokenzier by `padding=True`)
            for i, label_list in enumerate(label_batch):
                for _ in range(max_len - len(label_list)):
                    label_batch[i].append(0)
            label_batch=torch.tensor(label_batch).to(device)

            # Get model output
            outputs = model(encoding.input_ids, encoding.attention_mask, labels=label_batch)
            # Calculate loss and gradients
            loss = outputs.loss
            loss.backward()
            # Update model parameter
            optimizer.step()
            # Clear gradients (important!)
            model.zero_grad()
            # Reset batch-related vairiables
            text_batch = []
            label_batch = []
            max_len = 0
        
        # Run on validation set and report performance
        if (sent_cnt % report_every) == 0 or sent_cnt == len(training_data)-1:
            model.eval()
            correct_token, pred_token, true_token = 0.0, 0.0, 0.0
            print('Epoch {0}: trained {1} sentences'.format(epoch, sent_cnt))
            
            val_sent_cnt = 0
            val_text_batch = []
            val_label_batch = []
            val_max_len = 0

            for _, targets, full_sent in dataset_subword.data['valid_subword']:
                val_sent_cnt += 1
                # Accumulate sentences and label sequences for the current batch
                val_text_batch.append(full_sent)
                val_label_batch.append(targets)
                val_max_len = max(val_max_len, len(targets))

                if len(val_text_batch) == validation_batch_size or val_sent_cnt == len(dataset_subword.data['valid_subword'])-1:
                    # Tokenize the batch of input sentences
                    encoding = tokenizer(val_text_batch, return_tensors='pt', add_special_tokens=False, padding=True, truncation=True).to(device)
                    for i, label_list in enumerate(val_label_batch):
                        for _ in range(val_max_len - len(label_list)):
                            val_label_batch[i].append(0)
                    torch.tensor(val_label_batch)
                    val_label_batch=torch.tensor(val_label_batch).to(device)
                    outputs = model(**encoding)

                    # Go from logits to predicted labels by taking argmax
                    predictions = torch.argmax(outputs.logits, dim=2)
                    correct_token += torch.sum((val_label_batch != 0) & (predictions == val_label_batch))
                    pred_token += torch.sum((val_label_batch != 0))
                    true_token += torch.sum(val_label_batch != 0)
                    # Reset batch running variables
                    val_text_batch = []
                    val_label_batch = []
                    val_max_len = 0

            # After seeing all sentences in validation set, calculate precision, recall, F1
            precision = correct_token / pred_token if pred_token else 0.0
            recall = correct_token / true_token if true_token else 0.0
            f1 = 2 * precision * recall / (precision + recall) if (precision + recall) else 0.0
            print('Epoch {0}:\t validation precision\t {1:.1%}; recall\t {2:.1%}; F1\t {3:.1%}.'.format(epoch, precision, recall, f1))
            model.train()
    print()